# Ler pacotes

In [13]:
import os

In [14]:
current_path = os.getcwd()
if current_path.split('/')[-1] == 'content':
  %pip install trajminer -q

  from google.colab import drive

  drive.mount('/content/drive')
  print(f"current path:{current_path}. Moving to the project path.")
  os.chdir("drive/MyDrive/Colab Notebooks/mat_tree/")
  print(f"Current path: {os.getcwd()}")
else:
  print("Working locally.")
  print(f"current path:{current_path}.")

Working locally.
current path:c:\Users\yurin\Documents\github\mat_tree\preprocessing.


In [15]:
import pandas as pd
import numpy as np
import plotly.express as px


# Load data

## basometro

In [16]:
dtypes = {'UF':'string', 'voto':'string', 'orientacaoGoverno':'string', 'anoProposicao':'int32',
          'anoVotacao':'int32', 'tipoProposicao':'string', 'governo':'string', 'parlamentar':'string',
          # 'data':'datetime64[ns]',
          'idVotacao':'string', 'tid':'int64', 'label':'string', 'horaVotacao':'int32', 'diaDaSemanaVotacao':'int32', 'diaDaSemanaVotacaoNome':'string', 'diaDoAno':'int32', 'diaDoMesVotacao':'int32', 'mesVotacao': 'int32', 'delayVotacaoAnos':'int32', 'alinhamento':'string'}
dataset_path = "../datasets/basometro/"

basometro_df = pd.read_csv(dataset_path+'basometro_v1.csv', encoding='utf-8', delimiter=',', dtype=dtypes,
                           parse_dates=['data'], date_format="mixed", dayfirst=True)
basometro_df.head(3)

,UF,voto,orientacaoGoverno,anoProposicao,anoVotacao,tipoProposicao,governo,parlamentar,data,idVotacao,tid,partido,horaVotacao,diaDaSemanaVotacao,diaDaSemanaVotacaoNome,diaDoAno,diaDoMesVotacao,mesVotacao,delayVotacaoAnos,alinhamento
0,PR,Sim,Sim,2002,2003,MSC,Lula_1,Abelardo Lupion,2003-02-25 18:54:00,25-2-2003.18.54.736,5318,DEM,18,1,Tuesday,56,25,2,1,votou com o governo
1,PR,Sim,Sim,2002,2003,MPV,Lula_1,Abelardo Lupion,2003-03-18 19:30:00,18-3-2003.19.30.742,5318,DEM,19,1,Tuesday,77,18,3,1,votou com o governo
2,PR,Sim,Não,1996,2003,PL,Lula_1,Abelardo Lupion,2003-03-19 18:37:00,19-3-2003.18.37.743,5318,DEM,18,2,Wednesday,78,19,3,7,votou contra o governo


In [17]:
# basometro_df.info()

In [18]:
basometro_df['governo'].unique()

<StringArray>
['Lula_1', 'Lula_2', 'Dilma_1', 'Temer_1', 'Dilma_2', 'Bolsonaro_1']
Length: 6, dtype: string

In [19]:
# tmp = basometro_df[['tipoProposicao','data','idVotacao']]
# print(tmp.shape)
# tmp = tmp.drop_duplicates(subset=['idVotacao'])
# print(tmp.shape)
# tmp.head()
# tmp.to_csv('basometro_idVotacao.csv', index=False, header=True, encoding='utf-8')
# del tmp

## Political spectrum

In [20]:
df_info_esp_pol = pd.read_excel('info_esp_pol.xlsx', sheet_name='dados_esp_pol', header=0, dtype='string')
df_info_esp_pol.head(5)

,nome_partido,sigla,espectro_politico,posicionamento_ideologico,obs_dissolucao
0,Agir,AGIR,Centro,Conservador,<NA>
1,Avante,Avante,Centro,Conservador,<NA>
2,Cidadania,Cidadania,Centro-esquerda,Progressista,<NA>
3,Democracia Cristã,DC,Centro-direita,Conservador,<NA>
4,Democratas,DEM,Centro-direita,Conservador,1986 a 2022


# Preprocessing

## Help funcs

In [21]:
def convert_idVotacao(x):
    """Método para extrair o ano presente no idVotacao"""
    try:
        return int(x['idVotacao'].split(".")[0].split("-")[-1])
    except:
        return int(x['idVotacao'].split(".")[-1])

In [22]:
def get_periodo_mandato(ano_votacao):
    """Método para saber o período de mandado eleito com base no ano da votação."""
    if (ano_votacao >= 2003) and (ano_votacao <= 2006):
        return "2003-2006"
    elif (ano_votacao >= 2007) and (ano_votacao <= 2010):
        return "2007-2010"
    elif (ano_votacao >= 2011) and (ano_votacao <= 2014):
        return "2011-2014"
    elif (ano_votacao >= 2015) and (ano_votacao <= 2018):
        return "2015-2018"
    elif (ano_votacao >= 2019) and (ano_votacao <= 2022):
        return "2019-2022"
    else:
        return "outros"

In [23]:
def get_espetro_politico(x):
    """Método para retornar o espetro político"""
    try:
        return df_info_esp_pol[df_info_esp_pol['sigla'] == x]['espectro_politico'].iloc[0]
    except:
        return 'Indefinido'

# get_espetro_politico('Avane')

## Cleaning dataset

In [24]:
# fixing typos
basometro_df['partido'].replace({'NOVO':'Novo'}, inplace=True)
basometro_df['partido'].replace({'REDE':'Rede'}, inplace=True)

# dropping bad tids
print(basometro_df.shape)
basometro_df = basometro_df[basometro_df['tid'] != 5719]
print(basometro_df.shape)

C:\Users\yurin\AppData\Local\Temp\ipykernel_7104\951616392.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  basometro_df['partido'].replace({'NOVO':'Novo'}, inplace=True)


(1048575, 20)
(1048564, 20)


## Add political spectrum

In [25]:
basometro_df['esp_pol_partido'] = basometro_df['partido'].apply(lambda x: get_espetro_politico(x))
basometro_df.head(3)

,UF,voto,orientacaoGoverno,anoProposicao,anoVotacao,tipoProposicao,governo,parlamentar,data,idVotacao,...,partido,horaVotacao,diaDaSemanaVotacao,diaDaSemanaVotacaoNome,diaDoAno,diaDoMesVotacao,mesVotacao,delayVotacaoAnos,alinhamento,esp_pol_partido
0,PR,Sim,Sim,2002,2003,MSC,Lula_1,Abelardo Lupion,2003-02-25 18:54:00,25-2-2003.18.54.736,...,DEM,18,1,Tuesday,56,25,2,1,votou com o governo,Centro-direita
1,PR,Sim,Sim,2002,2003,MPV,Lula_1,Abelardo Lupion,2003-03-18 19:30:00,18-3-2003.19.30.742,...,DEM,19,1,Tuesday,77,18,3,1,votou com o governo,Centro-direita
2,PR,Sim,Não,1996,2003,PL,Lula_1,Abelardo Lupion,2003-03-19 18:37:00,19-3-2003.18.37.743,...,DEM,18,2,Wednesday,78,19,3,7,votou contra o governo,Centro-direita


## Feature engineering: alignment intra-party

### Calculate the total number of deputies per party during the presidential term.

In [26]:
tmp = basometro_df[['partido','anoVotacao','parlamentar']].copy()
tmp['periodo_mandato'] = tmp.apply(lambda x: get_periodo_mandato(x['anoVotacao']),axis=1)
tmp = tmp.sort_values(by=['partido','anoVotacao'])
# display(tmp.head(3))

### Calculando o atributo qtd_parlamentares_partido.
### Com a agregação abaixo nós queremos saber quantos parlamentares eleitos cada partido tinha em cada período de mandato.
df_tamanho_partidos = tmp.drop_duplicates(subset=['partido','parlamentar','periodo_mandato'])[['partido','periodo_mandato','parlamentar']] \
   .groupby(by=['partido','periodo_mandato']).agg(qtd_parlamentares_partido = ('parlamentar','count')).reset_index()
display(df_tamanho_partidos.head())

,partido,periodo_mandato,qtd_parlamentares_partido
0,Avante,2007-2010,1
1,Avante,2011-2014,2
2,Avante,2015-2018,7
3,Avante,2019-2022,8
4,Cidadania,2003-2006,33


In [27]:
# del tmp

### transformar df em dict para acelerar consulta
dict_tamanho_partidos = {}
for i, row in df_tamanho_partidos.iterrows():
    if row['partido'] in dict_tamanho_partidos:
        dict_tamanho_partidos[row['partido']][row['periodo_mandato']] = row['qtd_parlamentares_partido']
    else:
        dict_tamanho_partidos.update({row['partido']:{}})
        dict_tamanho_partidos[row['partido']][row['periodo_mandato']] = row['qtd_parlamentares_partido']
# del df_tamanho_partidos

### Calculate the number of votes by Party, VoteID and Proposition.

The vote type can be: Yes (Sim), No (Não), Obstruction (Obstrução), Abstention (Abstenção) and Absent (Ausente).

In [28]:

df_clean_tmp = basometro_df.drop_duplicates(subset=['idVotacao','parlamentar'])

tmp_gp = df_clean_tmp.groupby(['partido','idVotacao','tipoProposicao','voto']).agg(count = ('voto','count')) \
         .reset_index() \
         .sort_values(by=['partido','idVotacao','tipoProposicao'])

print(tmp_gp.shape)
tmp_gp.head(10)

(99073, 5)


,partido,idVotacao,tipoProposicao,voto,count
0,Avante,1-10-2019.00.20.16943,PL,Não,2
1,Avante,1-12-2021.17.21.17189,PL,Sim,1
2,Avante,1-12-2021.17.39.17189,PL,Sim,1
3,Avante,1-12-2021.17.54.17189,PL,Sim,1
4,Avante,1-4-2009.16.30.3673,MPV,Não,1
5,Avante,1-4-2009.19.23.3673,MPV,Sim,1
6,Avante,1-7-2021.12.04.17120,PL,Sim,1
7,Avante,1-7-2021.12.24.17120,PL,Sim,1
8,Avante,1-7-2021.13.22.17120,PL,Sim,2
9,Avante,1-8-2017.18.14.16274,MPV,Não,1


Pivot the groupby result to then calculate the proportions of Yes and No votes for the party.

In [29]:
table = pd.pivot_table(tmp_gp, values=['count'], index=['partido','idVotacao','tipoProposicao'], columns=['voto'],
                       aggfunc='sum', fill_value=0).reset_index().reset_index(drop=True).values

table = pd.DataFrame(table, columns=['partido','idVotacao','tipoProposicao','Abstenção','Obstrução','Não','AUSENTE','Sim'])
table['ano'] = table.apply(lambda x: convert_idVotacao(x),axis=1)
reorder_cols = ['partido','ano','idVotacao','tipoProposicao','Abstenção','Obstrução','AUSENTE','Não','Sim']
table = table[reorder_cols].sort_values(by=['partido','ano'])

print(table.shape)
display(table.head(5))
display(table[table['partido']=='Avante'].tail(5))

(69911, 9)


,partido,ano,idVotacao,tipoProposicao,Abstenção,Obstrução,AUSENTE,Não,Sim
4,Avante,2009,1-4-2009.16.30.3673,MPV,0,0,0,1,0
5,Avante,2009,1-4-2009.19.23.3673,MPV,0,0,0,0,1
10,Avante,2009,1-9-2009.17.17.3812,MPV,0,0,0,1,0
37,Avante,2009,10-11-2009.18.25.3883,MPV,0,0,0,1,0
38,Avante,2009,10-11-2009.19.54.3883,MPV,0,0,0,1,0


,partido,ano,idVotacao,tipoProposicao,Abstenção,Obstrução,AUSENTE,Não,Sim
1934,Avante,2022,9-2-2022.20.50.17206,PL,0,0,0,4,0
1935,Avante,2022,9-2-2022.21.35.17206,MPV,0,0,0,3,0
1936,Avante,2022,9-3-2022.21.04.17215,REQ,0,0,0,0,4
1937,Avante,2022,9-3-2022.22.02.17215,PEC,0,0,0,0,4
1938,Avante,2022,9-3-2022.22.11.17215,PEC,0,0,0,0,4


In [30]:
# del tmp_gp

Inserting attributes to a new table

In [31]:
lambda_ = lambda x: get_periodo_mandato(convert_idVotacao(x))
table['periodo_mandato'] = table.apply(lambda_, axis=1)
table['qtd_parlamentares_partido'] = table.apply(lambda x: dict_tamanho_partidos[x['partido']][x['periodo_mandato']], axis=1)

lambda_ = lambda x: x['qtd_parlamentares_partido']-(x['Abstenção']+x['Obstrução']+x['Não']+x['Sim'])
table['ausente_calculado'] = table.apply(lambda_, axis=1)

table['total_votos'] = table.apply(lambda x: x['Abstenção']+x['Obstrução']+x['Não']+x['Sim'], axis=1)

lambda_ = lambda x: np.round((x['Sim'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_sim_%'] = table.apply(lambda_ ,axis=1)

lambda_ = lambda x: np.round((x['Não'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_nao_%'] = table.apply(lambda_ ,axis=1)

lambda_ = lambda x: np.round((x['Abstenção'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_abstencao_%'] = table.apply(lambda_ ,axis=1)

lambda_ = lambda x: np.round((x['Obstrução'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_obstrucao_%'] = table.apply(lambda_ ,axis=1)

lambda_ = lambda x: np.round((x['ausente_calculado'])/x['qtd_parlamentares_partido']*100,2) if x['qtd_parlamentares_partido']!=0 else 0
table['partido_voto_ausente_%'] = table.apply(lambda_ ,axis=1)

reorder_cols = ['partido','idVotacao','ano','periodo_mandato','tipoProposicao','Abstenção','Obstrução','AUSENTE',
                'ausente_calculado','Não', 'Sim','total_votos','qtd_parlamentares_partido','partido_voto_sim_%','partido_voto_nao_%', 'partido_voto_abstencao_%', 'partido_voto_obstrucao_%', 'partido_voto_ausente_%']

new_table = table[reorder_cols].copy()
# new_table.head()
new_table.tail()

,partido,idVotacao,ano,periodo_mandato,tipoProposicao,Abstenção,Obstrução,AUSENTE,ausente_calculado,Não,Sim,total_votos,qtd_parlamentares_partido,partido_voto_sim_%,partido_voto_nao_%,partido_voto_abstencao_%,partido_voto_obstrucao_%,partido_voto_ausente_%
69906,União,7-4-2022.13.20.17230,2022,2019-2022,PL,0,0,0,55,29,2,31,86,2.33,33.72,0.0,0.00,63.95
69907,União,8-3-2022.20.26.17214,2022,2019-2022,PLP,0,0,0,30,15,41,56,86,47.67,17.44,0.0,0.00,34.88
69908,União,9-3-2022.21.04.17215,2022,2019-2022,REQ,0,1,0,26,7,52,60,86,60.47,8.14,0.0,1.16,30.23
69909,União,9-3-2022.22.02.17215,2022,2019-2022,PEC,0,1,0,36,2,47,50,86,54.65,2.33,0.0,1.16,41.86
69910,União,9-3-2022.22.11.17215,2022,2019-2022,PEC,0,1,0,29,1,55,57,86,63.95,1.16,0.0,1.16,33.72


In [32]:
drop_cols = ['partido', 'idVotacao', 'ano', 'periodo_mandato', 'tipoProposicao', 'Abstenção', 'Obstrução', 'AUSENTE',
             'ausente_calculado', 'Não', 'Sim']
new_table.drop(columns=drop_cols).sort_values(by=['total_votos'],ascending=False).head()

,total_votos,qtd_parlamentares_partido,partido_voto_sim_%,partido_voto_nao_%,partido_voto_abstencao_%,partido_voto_obstrucao_%,partido_voto_ausente_%
54512,92,107,83.18,2.80,0.0,0.00,14.02
54511,92,107,85.98,0.00,0.0,0.00,14.02
54345,91,107,74.77,2.80,0.0,7.48,14.95
54514,91,107,0.00,82.24,0.0,2.80,14.95
54242,90,107,81.31,2.80,0.0,0.00,15.89


We infer the party's orientation on a vote by being the highest proportion of votes it received. For example, if a Proposition had the highest proportion of YES votes, then the party's orientation is inferred as voting YES.

In [33]:
def get_orientacao_partido_voto(x):
    votos = ['partido_voto_sim_%', 'partido_voto_nao_%', 'partido_voto_abstencao_%', 'partido_voto_obstrucao_%',
                   'partido_voto_ausente_%']
    map_name = {'sim':'Sim', 'nao':'Não', 'abstencao':'Abstenção', 'obstrucao':'Obstrução', 'ausente':'Ausente'}
    max_proporcao  = 0
    max_voto = ''

    for voto in votos:
        if x[voto] > max_proporcao:
            max_voto = voto.split('_')[-2]
            max_proporcao = x[voto]
    
    return map_name[max_voto]

In [34]:
new_table['orientacao_partido_voto'] = new_table.apply(lambda x: get_orientacao_partido_voto(x), axis=1)

new_table.drop(columns=drop_cols).tail()

,total_votos,qtd_parlamentares_partido,partido_voto_sim_%,partido_voto_nao_%,partido_voto_abstencao_%,partido_voto_obstrucao_%,partido_voto_ausente_%,orientacao_partido_voto
69906,31,86,2.33,33.72,0.0,0.00,63.95,Ausente
69907,56,86,47.67,17.44,0.0,0.00,34.88,Sim
69908,60,86,60.47,8.14,0.0,1.16,30.23,Sim
69909,50,86,54.65,2.33,0.0,1.16,41.86,Sim
69910,57,86,63.95,1.16,0.0,1.16,33.72,Sim


In [35]:
### criando dict do table para preencher o dataset do basomentro com as novas infos
info_orientacao_partido = {}
cols_votos = ['partido_voto_sim_%','partido_voto_nao_%','partido_voto_abstencao_%','partido_voto_obstrucao_%',
              'partido_voto_ausente_%','orientacao_partido_voto']

for index, row in new_table.iterrows():
    if row['partido'] not in info_orientacao_partido.keys():
        info_orientacao_partido.update({row['partido']:{}})
    
    if row['idVotacao'] in info_orientacao_partido[row['partido']]:
        if row['tipoProposicao'] in info_orientacao_partido[row['partido']][row['idVotacao']]:
            for col in cols_votos:
                if col not in info_orientacao_partido[row['partido']][row['idVotacao']][row['tipoProposicao']]:
                    info_orientacao_partido[row['partido']][row['idVotacao']][row['tipoProposicao']] = {col: row[col]}
        else:# acho que nao precisa
            info_orientacao_partido[row['partido']][row['idVotacao']] = {row['tipoProposicao']:{}}
            for col in cols_votos:
                if col not in info_orientacao_partido[row['partido']][row['idVotacao']][row['tipoProposicao']]:
                    info_orientacao_partido[row['partido']][row['idVotacao']][row['tipoProposicao']][col] = row[col]
    else:
        info_orientacao_partido[row['partido']][row['idVotacao']] = {}
        info_orientacao_partido[row['partido']][row['idVotacao']][row['tipoProposicao']] = {}

        for col in cols_votos:
                if col not in info_orientacao_partido[row['partido']][row['idVotacao']][row['tipoProposicao']]:
                    info_orientacao_partido[row['partido']][row['idVotacao']][row['tipoProposicao']][col] = row[col]

# del table

In [36]:
# info_orientacao_partido
info_orientacao_partido['União']['7-4-2022.13.20.17230']

{'PL': {'partido_voto_sim_%': 2.33,
  'partido_voto_nao_%': 33.72,
  'partido_voto_abstencao_%': 0.0,
  'partido_voto_obstrucao_%': 0.0,
  'partido_voto_ausente_%': 63.95,
  'orientacao_partido_voto': 'Ausente'}}

### Concatenate Government + Alignment aspects

In [37]:
basometro_df['governo_alinhamento'] = basometro_df['governo']+'/'+basometro_df['alinhamento']
basometro_df.head(2)

,UF,voto,orientacaoGoverno,anoProposicao,anoVotacao,tipoProposicao,governo,parlamentar,data,idVotacao,...,horaVotacao,diaDaSemanaVotacao,diaDaSemanaVotacaoNome,diaDoAno,diaDoMesVotacao,mesVotacao,delayVotacaoAnos,alinhamento,esp_pol_partido,governo_alinhamento
0,PR,Sim,Sim,2002,2003,MSC,Lula_1,Abelardo Lupion,2003-02-25 18:54:00,25-2-2003.18.54.736,...,18,1,Tuesday,56,25,2,1,votou com o governo,Centro-direita,Lula_1/votou com o governo
1,PR,Sim,Sim,2002,2003,MPV,Lula_1,Abelardo Lupion,2003-03-18 19:30:00,18-3-2003.19.30.742,...,19,1,Tuesday,77,18,3,1,votou com o governo,Centro-direita,Lula_1/votou com o governo


## Fill data into the dataset

In [38]:
def get_tipoVotoPartido(x, tipoVotoPartido):
    return info_orientacao_partido[x['partido']][x['idVotacao']][x['tipoProposicao']][tipoVotoPartido]

In [39]:
basometro_df['voto'].unique()

<StringArray>
['Sim', 'Não', 'Obstrução', 'Abstenção', 'AUSENTE']
Length: 5, dtype: string

In [40]:
print("Inserindo atributos sobre proporção de vatos e orientação partido.")
for tipoVoto in cols_votos:
    print(f"\t Tipo voto: '{tipoVoto}'...",end='')
    lambda_ = lambda x: get_tipoVotoPartido(x, tipoVoto)
    basometro_df[tipoVoto] = basometro_df.apply(lambda_, axis=1)
    print("DONE")

print("Criando atributo 'alinhamento_candidato'...",end='')
lambda_ = lambda x: 'votou com o partido' if x['orientacao_partido_voto'].lower() == x['voto'].lower() else 'votou contra o partido'
basometro_df['alinhamento_candidato'] = basometro_df.apply(lambda_, axis=1)
print("DONE")

basometro_df.head(2)

Inserindo atributos sobre proporção de vatos e orientação partido.
	 Tipo voto: 'partido_voto_sim_%'...DONE
	 Tipo voto: 'partido_voto_nao_%'...DONE
	 Tipo voto: 'partido_voto_abstencao_%'...DONE
	 Tipo voto: 'partido_voto_obstrucao_%'...DONE
	 Tipo voto: 'partido_voto_ausente_%'...DONE
	 Tipo voto: 'orientacao_partido_voto'...DONE
Criando atributo 'alinhamento_candidato'...DONE


,UF,voto,orientacaoGoverno,anoProposicao,anoVotacao,tipoProposicao,governo,parlamentar,data,idVotacao,...,alinhamento,esp_pol_partido,governo_alinhamento,partido_voto_sim_%,partido_voto_nao_%,partido_voto_abstencao_%,partido_voto_obstrucao_%,partido_voto_ausente_%,orientacao_partido_voto,alinhamento_candidato
0,PR,Sim,Sim,2002,2003,MSC,Lula_1,Abelardo Lupion,2003-02-25 18:54:00,25-2-2003.18.54.736,...,votou com o governo,Centro-direita,Lula_1/votou com o governo,55.67,0.0,0.0,0.0,44.33,Sim,votou com o partido
1,PR,Sim,Sim,2002,2003,MPV,Lula_1,Abelardo Lupion,2003-03-18 19:30:00,18-3-2003.19.30.742,...,votou com o governo,Centro-direita,Lula_1/votou com o governo,52.58,13.4,0.0,0.0,34.02,Sim,votou com o partido


In [41]:
print(f"basometro no-clean shape: {basometro_df.shape}")
print(f"basometro clean shape: {basometro_df.drop_duplicates(subset=['partido','idVotacao','parlamentar']).shape}")
print("Saving dataset in CSV format with new features ...",end='')
basometro_df.drop_duplicates(subset=['partido','idVotacao','parlamentar']) \
            .to_csv(dataset_path+'basometro_v2.csv', encoding='utf-8', index=False, header=True)
print("DONE")

basometro no-clean shape: (1048564, 29)
basometro clean shape: (1019353, 29)
Salvando dataset no formato CSV com novas features...DONE
